##SAI SRIKANTH SARABU
##A20343781

In [1]:
!pip install pyspark

     |████████████████████████████████| 212.4 MB 68 kB/s 
     |████████████████████████████████| 198 kB 60.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=b6e86d4bbaf22263808ef2a7b2cb29bada08356d4c391cc9d4f71363a711a1b4
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import sys
from pyspark import SparkContext, SparkConf

In [4]:
sc = SparkContext("local","Project-1: A Simple Movie Recommender with Distributed Association Rules Analysis")

In [5]:
#All file Paths
file1 = '/content/drive/MyDrive/Sample Data/views.txt'
file2 = '/content/drive/MyDrive/Sample Data/mappings.txt'
file3 = '/content/drive/MyDrive/Sample Data/get_recommendations.txt'

In [6]:
file =  sc.textFile(file1) 
completedata = file.map(lambda line:frozenset(line.split('::')))  #getting each line from the file and making it to frozenset
file.unpersist()
completedata.persist()

PythonRDD[2] at RDD at PythonRDD.scala:53

#Generating frequent items


In [7]:
s=3  #Initializing the support value

#below line will generate frequent singletons 
singletons = file.flatMap(lambda line:line.split('::')).map(lambda x: frozenset((x,))).map(lambda key: (key , 1)).reduceByKey(lambda a,b: a+b).filter(lambda a: a[1]>=s)

singletons.persist() #persisting the singleton data
#singletons.take(5)

PythonRDD[7] at RDD at PythonRDD.scala:53

In [8]:
#the below function is used to generate n-sized combinations of given list x, returns list of lists of combinations
def getCombs(x,n):
  mylist =[]
  if n==0:
    return [mylist]
  xlen = len(x)
  for j in range(xlen): 
    klist = getCombs(x[j + 1:xlen], n-1) 
    #print(klist)
    for i in klist:
      mylist.append(list([x[j]])+i)
  return mylist


In [9]:
#below function is used for checking each item in previous frequent RDD with each line in complete data, 
#and making combinations with those items which are present.
def combs(x,freqdata,k):  #here x is a line from complete data
  mylist= []
  a= frozenset()
  for i in freqdata.value:   # looping through each item previous frequent data
    if set(i).issubset(x):   # checking whether an item is present in line of complete data or not 
      a = a.union(i)         # preparing a list which are present
  mylist = getCombs(list(a),k) # and making freq pairs, triplets, and quadruplets from the list of frequently present items.
  
  return mylist 

In [10]:

def frequent_itemsets(prev,k,freqcands):
  prev = prev.keys()                              
  prevbroad = sc.broadcast(prev.collect())      #broadcasting the previous frequent candidate items
  combinedData=completedata.filter(lambda x: len(x)>=k)  #checking if each line length is greater than or equal to k for making combinations with k
  combinedData = combinedData.flatMap(lambda x: combs(x,prevbroad,k))  # calling function combs with each line from complete data
  #making the freq pairs, triplets or quadruplets to key and value with support 
  combinedData = combinedData.map(lambda x:(frozenset(x),1)).reduceByKey(lambda a,b: a+b).filter(lambda a: a[1]>=s)
  freqcands = freqcands.union(combinedData)  #combining all the previous data with the latest
  if k<=3:
    return frequent_itemsets(combinedData,k+1,freqcands)  # calling the same function again by increasing k value.
  return freqcands   

In [11]:
freqcands= sc.parallelize([])
freqcands=frequent_itemsets(singletons,2,freqcands)  # calling the function frequent_itemsets
#freqcands.count()
freqcands = singletons.union(freqcands) #combining all singletons, pairs, triplets and quadruplets data
singletons.unpersist()  #unpersisting the data as it is not required


PythonRDD[7] at RDD at PythonRDD.scala:53

In [12]:

dic = {}
for d in freqcands.map(lambda x: dict([x])).collect(): # here we are making dictionary of freq items and support value
  dic.update(d)

freq_dic = sc.broadcast(dic) #broadcasting the dictionary

#Generating association rules

In [13]:
con_thres= 60 #initializing the confidence threshold value

#below function is used to return antecedent, consequent and confidence of the association rules
def assoc_rules(x):
  x = x[0]
  if len(x) == 1: #here we are avoiding sigletons as we need only higher order to generate association rules
    return ''
  else:
    sup_num = freq_dic.value[x] #numerator of the confidence would be support value of all the elements together occuring
    mylist = list(x)            #the above line is using freq_dic to get the support value
    clist = []
    for i in mylist:         # in this loop we are removing(this will be our consequent element) an each item once in iteration and making rules based on that. 
      tempList = mylist.copy() # making an copy of list
      tempList.remove(i)       #removing an item in iteration
      sup_den = freq_dic.value[frozenset(tempList)]  #getting support of antecedent elements
      con = round((sup_num/sup_den)*100,2)  #finding confidence
      if con >= con_thres:                  #checking whether it is present in threshold limit or not
        clist.append((tempList,i,con))      #appending antecedent, consequent elements and confidence of association rule to a list

  return clist

In [14]:
association_rules = freqcands.flatMap(lambda x:assoc_rules(x)) # calling function assoc_rules(x), x being each frequent item.
freqcands.unpersist()  #unpersisting the freqcands as we have no use further

UnionRDD[31] at union at NativeMethodAccessorImpl.java:0

In [15]:
gmap =  sc.textFile(file2) #file 2 is mapping data for each id in complete data
mapping = gmap.map(lambda line:line.split('::')) 
maping = {}
for d in mapping.map(lambda x: dict([x])).collect(): # here we are making mapping into key value pairs using dictionary
  maping.update(d)
mappingsdict = sc.broadcast(maping) #broadcasting the mapping dictionary
gmap.unpersist()  #unpersisting the data, which are not required
mapping.unpersist()


# making this dictionary to use in the bonus task
bonusDict = {}
#making a dictionary with key as antecedent and value as consequent and confidence
for i, *rest in association_rules.map(lambda x: (frozenset(x[0]),(x[1],x[2]))).collect(): 
  bonusDict.setdefault(i, []).extend(rest)


#the function below will map value of id with its movie name and return
#here we are returning coma separated antecedent, consequent and confidence 
def get_res(x):
  cList = []
  b = mappingsdict.value[x[1]]
  a = ''
  for j in x[0]:
    a+=', '+mappingsdict.value[j]
  cList.append((a,b,x[2]))
  return cList

#the below line is formating the output and making confidence as key
association_rules = association_rules.flatMap(lambda x:get_res(x)).map(lambda x : (x[2],x[0].strip(',') + '  ->  '+  x[1] +' ; Confidence = '+ str(x[2]) + ' % '))
association_rules = association_rules.sortByKey(False).map(lambda x:x[1])#here we are sorting the elements in decreasing order of confidence
association_rules.coalesce(1).saveAsTextFile('output.txt') #using coalesce to generate single file as output of association rules

#Bonus Task 

In [16]:
count = 1 # initializing line count

def bon(x):#here x is the list of single watch history
  myList = []

  if len(x)>=3:
    for j in getCombs(x,3):#below 3 loops is generating combinations of watch history of size 1,2 and 3 as we have only antecedent size upto 3 in association rules 
      myList.append(j)
  if len(x)>=2:
    for o in getCombs(x,2):
      myList.append(o) 
  if len(x)>=1:
    for l in getCombs(x,1):
      myList.append(l)
  
  cList = []
  for i in myList:                #here we are looping through antecedent values and checking for a hit, and getting the recommendation as consequent
    for r in bonusDict.keys():    #looping through all the antecedent values
      if len(i) <= len(r):        #checking if length of the an item is less than antecedent value
        if set(i).issubset(r):    #checking if it subset or not
          cList.append(bonusDict[r])  #appending the consequent and confidence value, as value contains both
          
  global count
  co = count
  count+=1
  if len(cList) > 0:
    flat_list = [i for slist in cList for i in slist] #making list of lits to list, as each consequent and confidence is a list
    flat_list.sort(key = lambda x: x[1], reverse = True) #arraning tupples in list in decreasing order
    result = []
    if len(cList)>=3:     #here we are recommenting top 3 movies in highest confidence
      for i in range(3):
        result.append(mappingsdict.value[flat_list[i][0]]) #flat list will give us id, based on that mapping dictionary will change id to movie name
    else:
      for i in range(len(cList)): #if we get less than 3 recommendations 
        result.append(mappingsdict.value[flat_list[i][0]])
    return ('line '+ str(co) ,result)
  else: # if we do not have any recommendations from the association rules
    return 'line '+ str(co), 'No recommendation found from the above Association Rules'


bonus =  sc.textFile(file3) #file 3 contains 10 watch histories
bonus.map(lambda line:bon(line.split('::'))).collect() # calling the method bon with single watch history and k as parameter to get rcommendations


[('line 1',
  ['Airplane! (1980)',
   'American Beauty (1999)',
   'Star Trek V: The Final Frontier (1989)']),
 ('line 2', 'No recommendation found from the above Association Rules'),
 ('line 3', 'No recommendation found from the above Association Rules'),
 ('line 4', 'No recommendation found from the above Association Rules'),
 ('line 5', 'No recommendation found from the above Association Rules'),
 ('line 6',
  ['Cinderella (1950)',
   'Beauty and the Beast (1991)',
   'South Park: Bigger, Longer and Uncut (1999)']),
 ('line 7', 'No recommendation found from the above Association Rules'),
 ('line 8',
  ['Terminator 2: Judgment Day (1991)',
   'Jurassic Park (1993)',
   'Raiders of the Lost Ark (1981)']),
 ('line 9', 'No recommendation found from the above Association Rules'),
 ('line 10', ['Groundhog Day (1993)', 'Player, The (1992)'])]